In [1]:
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm

In [2]:
def clear_link(page_link):
    page_link = re.sub('^\/', '', page_link)
    page_link = re.sub('[\#\?].+', '', page_link)
    
    return page_link

In [5]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
# proxies = {"http": "http://177.93.72.98:4153"}


#функция получает html страницы, это стандартный способ получения с помощью requests
def get_page_text(url, user_agent, error_log=True):
    headers = {'User-Agent': user_agent}
    response = requests.get(url, headers=headers, allow_redirects=True) #, proxies=proxies

    if response.status_code == 200:
        page_text = response.text
        return page_text
    elif error_log:
        print(f'Ошибка! Response code: {response.status_code}')
        print(f'URL: {url}')
        print('--------------')
    
    return None

In [9]:
url = "https://cozyhome.ru/product/stino_723876#offer_109345"
page_text = get_page_text(url, USER_AGENT)

In [10]:
soup = BeautifulSoup(page_text) #Beautiful Soup is a Python library for pulling data out of HTML and XML files.
#https://www.crummy.com/software/BeautifulSoup/bs4/doc/ - документация

In [17]:
def get_page_links(soup, main_url):
    links = set()
    for a in soup.find_all('a'):
        if not a.has_attr('href'):
            continue

        page_link = a.attrs['href']

        if re.search('^(http)', page_link):
            if main_url not in page_link or main_url == page_link:
                continue

        page_link = clear_link(page_link)

        if len(page_link) == 0:
            continue 
        
        if main_url not in page_link:
            page_link = main_url + page_link

        links.add(page_link)
    
    return links

In [18]:
all_links = get_page_links(soup, url)
process_links = all_links
process_links

{'https://leroymerlin.ru/about/',
 'https://leroymerlin.ru/advice/',
 'https://leroymerlin.ru/basket/',
 'https://leroymerlin.ru/basket/regulations/',
 'https://leroymerlin.ru/biznes/',
 'https://leroymerlin.ru/brands/',
 'https://leroymerlin.ru/catalogue/',
 'https://leroymerlin.ru/catalogue/aerozoli-i-smazki/',
 'https://leroymerlin.ru/catalogue/aksessuary-dlya-bani-i-sauny/',
 'https://leroymerlin.ru/catalogue/aksessuary-dlya-elektroinstrumenta/',
 'https://leroymerlin.ru/catalogue/aksessuary-dlya-hraneniya/',
 'https://leroymerlin.ru/catalogue/aksessuary-dlya-vannoy-komnaty/',
 'https://leroymerlin.ru/catalogue/barbekyu-dlya-dachi/',
 'https://leroymerlin.ru/catalogue/barnye-stoyki-i-aksessuary/',
 'https://leroymerlin.ru/catalogue/biotualety-i-aksessuary/',
 'https://leroymerlin.ru/catalogue/bloki-dlya-stroitelstva/',
 'https://leroymerlin.ru/catalogue/bytovaya-tehnika-dlya-kuhni/',
 'https://leroymerlin.ru/catalogue/compare/',
 'https://leroymerlin.ru/catalogue/dekor-dlya-detskoy

In [14]:
def get_info(soup):
#     page_text = get_page_text(lnk, USER_AGENT)
#     soup = BeautifulSoup(page_text)
    name = soup.title.string
    characteristics = soup.find("div", class_="tabs__content _active").get_text().replace("\n", "") #id="nav-characteristics"
    price = soup.find("span", class_="_price sale_price").get_text().replace(" ", "").replace("₽", "")  #slot="price"
#     with open("characteristics.csv", 'a') as dst:
#         dst.write(f'{name},{characteristics},{price}\n')
    print(f'Наименование товара: {name}\n\n Характеристики: {characteristics}\n\n Цена {price}\n')

In [27]:
# Сбор всех ссылок с ресурса
while True:
    new_links = set()

    for lnk in tqdm(process_links):
        try:
            page_text = get_page_text(lnk, USER_AGENT)
            soup = BeautifulSoup(page_text)
            links = get_page_links(soup, url)
            links = links.difference(all_links, new_links)
            
            try:
                get_info(soup)
            except AttributeError as e:
                print(f"page {lnk} without product. Error {e}")

            new_links = new_links.union(links)
        except TypeError:
            print(f"{lnk} status 404")

    if len(new_links) == 0:
        break

    all_links = all_links.union(new_links)
    process_links = new_links
    
print(f"Обработано {len(all_links)} ссылок")

  0%|▏                                          | 1/330 [00:01<07:35,  1.39s/it]

page https://leroymerlin.ru/catalogue/elektroinstrumenty/ without product. Error 'NoneType' object has no attribute 'get_text'


  1%|▎                                          | 2/330 [00:02<07:03,  1.29s/it]

page https://leroymerlin.ru/catalogue/mikroklimat/ without product. Error 'NoneType' object has no attribute 'get_text'


  1%|▍                                          | 3/330 [00:03<06:20,  1.16s/it]

...

page https://leroymerlin.ru/catalogue/ without product. Error 'NoneType' object has no attribute 'get_text'


  2%|▉                                          | 7/330 [00:06<03:41,  1.46it/s]

Ошибка! Response code: 401
URL: https://leroymerlin.ru/product/salfetka-dlya-mebeli-i-tehniki-35h35-sm-82113286/
--------------
https://leroymerlin.ru/product/salfetka-dlya-mebeli-i-tehniki-35h35-sm-82113286/ status 404


  2%|▉                                          | 7/330 [00:08<06:29,  1.20s/it]


KeyboardInterrupt: 

Для получения информации с сайта необходимо знать селектор того блока, информацию о котором мы хотим получить. У одного сайта на разных страницах эти силекторы как правило одинаковые. Если нужно будет применить этот способ к другому сайту, то силектор придется менять

In [15]:
get_info(soup)

Наименование товара: Подсвечник Stino Розовый - купить по цене 1847 руб. в интернет-магазине COZY HOME

 Характеристики: Капсула:                                                     Raspberry Lipstick                                                     Состав: СтеклоЦвет: РозовыйРазмер: Размер наволочки, см: Размер простыни, см: Размер пододеяльника, см: Комплектация: Подсвечник- 1 шт.Вес, кг: 0.5Габариты, см: 9*9*35

 Цена 
1847

